# 'GATK resource bundle' on Azure Genomics Data Lake

# DATASET 1. Downloading 'broad-public-datasets'from Azure Genomics Data Lake


Jupyter notebooks are a great tool for data scientists who is working on Genomics data analysis. We will demonstrate how to download specific 'broad-public-datasets' from Azure Genomics Data Lake. 

**Here is the coverage of this notebook:**

1. Downloading genomics data
2. Annotate genotypes using VariantFiltration
3. Select Specific Variants
4. Convert gVCF file to a table 

**Dependencies:**

This notebook requires the following libraries:

- Azure storage `pip install azure-storage-blob==2.1.0`. Please visit [this page](https://github.com/Azure/azure-storage-python/wiki) for frequently encountered problem for this SDK.

- Genome Analysis Toolkit (GATK) (*Users need to download GATK from Broad Institute's webpage into the same compute environment with this notebook: https://github.com/broadinstitute/gatk/releases*)

**Important information: This notebook is using Python 3.6 kernel**


In [ ]:
pip install azure-storage-blob==2.1.0

### Getting the Genomics data from Azure Open Datasets

[Description of dataset](https://gatk.broadinstitute.org/hc/en-us/articles/360035890811-Resource-bundle): Stores public test data, often used to test workflows. For example, it contains NA12878 CRAM, gVCF, and unmapped BAM files.

### Downloading the specific 'broad-public-dataset'

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetbroadpublic', sas_token='sv=2020-04-08&si=prod&sr=c&sig=u%2Bg2Ab7WKZEGiAkwlj6nKiEeZ5wdoJb10Az7uUwis%2Fg%3D')     
blob_service_client.get_blob_to_path('dataset/NA12878', 'NA12878.g.vcf.gz', './NA12878.g.vcf.gz')

Downloading the the index file of gVCF file

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetbroadpublic', sas_token='sv=2020-04-08&si=prod&sr=c&sig=u%2Bg2Ab7WKZEGiAkwlj6nKiEeZ5wdoJb10Az7uUwis%2Fg%3D')     
blob_service_client.get_blob_to_path('dataset/NA12878', 'NA12878.g.vcf.gz.tbi', './NA12878.g.vcf.gz.tbi')

### Downloading `hg38` reference genome

Downloading the the reference genome bundle (hg38) from `public-broad-ref` on Azure Genomics Data Lake

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetpublicbroadref', sas_token='sv=2020-04-08&si=prod&sr=c&sig=DQxmjB4D1lAfOW9AxIWbXwZx6ksbwjlNkixw597JnvQ%3D')     
blob_service_client.get_blob_to_path('dataset/hg38/v0', 'Homo_sapiens_assembly38.fasta', './Homo_sapiens_assembly38.fasta')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetpublicbroadref', sas_token='sv=2020-04-08&si=prod&sr=c&sig=DQxmjB4D1lAfOW9AxIWbXwZx6ksbwjlNkixw597JnvQ%3D')     
blob_service_client.get_blob_to_path('dataset/hg38/v0', 'Homo_sapiens_assembly38.dict', './Homo_sapiens_assembly38.dict')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetpublicbroadref', sas_token='sv=2020-04-08&si=prod&sr=c&sig=DQxmjB4D1lAfOW9AxIWbXwZx6ksbwjlNkixw597JnvQ%3D')     
blob_service_client.get_blob_to_path('dataset/hg38/v0', 'Homo_sapiens_assembly38.fasta.fai', './Homo_sapiens_assembly38.fai')

### 1. Annotate genotypes using VariantFiltration

**Important note: Please check your GATK is running on your system.**

If we want to filter heterozygous genotypes, we use VariantFiltration's `--genotype-filter-expression isHet == 1` option. We can specify the annotation value for the tool to label the heterozygous genotypes with with the `--genotype-filter-name` option. Here, this parameter's value is set to `isHetFilter`. In our first example, we used `NA12877.vcf.gz` from Illimina Platinum Genomes but users can use any vcf files from other datasets:`Platinum Genomes`

In [ ]:
run gatk VariantFiltration -V NA12878.g.vcf.gz -O outputannot.vcf --genotype-filter-expression "isHet == 1" --genotype-filter-name "isHetFilter"

### 2. Select Specific Variants

Select a subset of variants from a VCF file
This tool makes it possible to select a subset of variants based on various criteria in order to facilitate certain analyses. Examples of such analyses include comparing and contrasting cases vs. controls, extracting variant or non-variant loci that meet certain requirements, or troubleshooting some unexpected results, to name a few.

There are many different options for selecting subsets of variants from a larger call set:

Extract one or more samples from a callset based on either a complete sample name or a pattern match.
Specify criteria for inclusion that place thresholds on annotation values, **e.g. "DP > 1000" (depth of coverage greater than 1000x), "AF < 0.25" (sites with allele frequency less than 0.25)**.These criteria are written as "JEXL expressions", which are documented in the article about using JEXL expressions.
Provide concordance or discordance tracks in order to include or exclude variants that are also present in other given callsets.
Select variants based on criteria like their type (e.g. INDELs only), evidence of mendelian violation, filtering status, allelicity, etc.
There are also several options for recording the original values of certain annotations which are recalculated when one subsets the new callset, trims alleles, etc.

**Input**

A variant call set in VCF format from which a subset can be selected.

**Output**

A new VCF file containing the selected subset of variants.

In [ ]:
run gatk SelectVariants -R Homo_sapiens_assembly38.fasta -V outputannot.vcf --select-type-to-include SNP --select-type-to-include INDEL -O selective.vcf

### 3. VariantsToTable

Extract fields from a VCF file to a tab-delimited table
This tool extracts specified fields for each variant in a VCF file to a tab-delimited table, which may be easier to work with than a VCF. By default, the tool only extracts PASS or . (unfiltered) variants in the VCF file. Filtered variants may be included in the output by adding the --show-filtered flag. The tool can extract both INFO (i.e. site-level) fields and FORMAT (i.e. sample-level) fields.


**INFO/site-level fields**

Use the `-F` argument to extract INFO fields; each field will occupy a single column in the output file. The field can be any standard VCF column (e.g. CHROM, ID, QUAL) or any annotation name in the INFO field (e.g. AC, AF). The tool also supports the following additional fields:

EVENTLENGTH (length of the event)
TRANSITION (1 for a bi-allelic transition (SNP), 0 for bi-allelic transversion (SNP), -1 for INDELs and multi-allelics)
HET (count of het genotypes)
HOM-REF (count of homozygous reference genotypes)
HOM-VAR (count of homozygous variant genotypes)
NO-CALL (count of no-call genotypes)
TYPE (type of variant, possible values are NO_VARIATION, SNP, MNP, INDEL, SYMBOLIC, and MIXED
VAR (count of non-reference genotypes)
NSAMPLES (number of samples)
NCALLED (number of called samples)
MULTI-ALLELIC (is this variant multi-allelic? true/false)


**FORMAT/sample-level fields**

Use the `-GF` argument to extract FORMAT/sample-level fields. The tool will create a new column per sample with the name "SAMPLE_NAME.FORMAT_FIELD_NAME" e.g. NA12877.GQ, NA12878.GQ.



**Input**

A VCF file to convert to a table

**Output**

A tab-delimited file containing the values of the requested fields in the VCF file.


In [ ]:
run gatk VariantsToTable -V NA12878.g.vcf.gz -F CHROM -F POS -F TYPE -F AC -F AD -F AF -GF DP -GF AD -O outputtable.table

### References

1. VariantFiltration: https://gatk.broadinstitute.org/hc/en-us/articles/360036827111-VariantFiltration 
2. Select Variants:https://gatk.broadinstitute.org/hc/en-us/articles/360037052272-SelectVariants
3. Concordance: https://gatk.broadinstitute.org/hc/en-us/articles/360041851651-Concordance
4. Variants to table: https://gatk.broadinstitute.org/hc/en-us/articles/360036882811-VariantsToTable 
5. Broad Resource bundle:https://gatk.broadinstitute.org/hc/en-us/articles/360035890811-Resource-bundle 



# DATASET 2. Downloading 'public-data-broad-references'from Azure Genomics Data Lake


We will demonstrate how to download specific 'public-data-broad-references' from Azure Genomics Data Lake. 



### Getting the Genomics data from Azure Open Datasets

[Description of dataset](https://gatk.broadinstitute.org/hc/en-us/articles/360035890811-Resource-bundle): This is the Broad's public hg38 and b37 reference and resource data. Additional information can be found in the GATK Resource Bundle article. This bucket is controlled by Broad, but hosted by `Microsoft`.

* Whole-Genome-Analysis-Pipeline
* GATK4-Germline-Preprocessing-VariantCalling-JointCalling

### Downloading `hg38` reference genome

Downloading the the reference genome bundle (hg38) from `public-data-broad-reference` on Azure Genomics Data Lake

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetpublicbroadref', sas_token='sv=2020-04-08&si=prod&sr=c&sig=DQxmjB4D1lAfOW9AxIWbXwZx6ksbwjlNkixw597JnvQ%3D')     
blob_service_client.get_blob_to_path('dataset/hg38/v0', 'Homo_sapiens_assembly38.fasta', './Homo_sapiens_assembly38.fasta')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetpublicbroadref', sas_token='sv=2020-04-08&si=prod&sr=c&sig=DQxmjB4D1lAfOW9AxIWbXwZx6ksbwjlNkixw597JnvQ%3D')     
blob_service_client.get_blob_to_path('dataset/hg38/v0', 'Homo_sapiens_assembly38.dict', './Homo_sapiens_assembly38.dict')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetpublicbroadref', sas_token='sv=2020-04-08&si=prod&sr=c&sig=DQxmjB4D1lAfOW9AxIWbXwZx6ksbwjlNkixw597JnvQ%3D')     
blob_service_client.get_blob_to_path('dataset/hg38/v0', 'Homo_sapiens_assembly38.fasta.fai', './Homo_sapiens_assembly38.fai')

# DATASET 3. Downloading 'gatk-legacy-bundles' from Azure Genomic Data Lake

We will demonstrate how to download specific 'gatk-legacy-bundles' from Azure Genomics Data Lake. 



### Getting the Genomics data from Azure Genomics Data Lake

[Description of dataset](https://gatk.broadinstitute.org/hc/en-us/articles/360035890811-Resource-bundle): Broad public legacy b37 and hg19 reference and resource data.

### Downloading the `ucsc.hg.19` data bundle from 'gatk-test-data'

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatklegacybundles', sas_token='sv=2020-04-08&si=prod&sr=c&sig=xBfxOPBqHKUCszzwbNCBYF0k9osTQjKnZbEjXCW7gU0%3D')     
blob_service_client.get_blob_to_path('dataset/hg19', 'ucsc.hg19.fasta', './ucsc.hg19.fasta')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatklegacybundles', sas_token='sv=2020-04-08&si=prod&sr=c&sig=xBfxOPBqHKUCszzwbNCBYF0k9osTQjKnZbEjXCW7gU0%3D')     
blob_service_client.get_blob_to_path('dataset/hg19', 'ucsc.hg19.fasta.fai', './ucsc.hg19.fasta.fai')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatklegacybundles', sas_token='sv=2020-04-08&si=prod&sr=c&sig=xBfxOPBqHKUCszzwbNCBYF0k9osTQjKnZbEjXCW7gU0%3D')     
blob_service_client.get_blob_to_path('dataset/hg19', 'ucsc.hg19.dict', './ucsc.hg19.dict')

# DATASET 4. Downloading 'gatk-best-practices' from Azure Genomic Data Lake

We will demonstrate how to download specific 'gatk-best-practices' from Azure Genomics Data Lake.


### Getting the Genomics data from Azure Genomics Data Lake

[Description of dataset](https://gatk.broadinstitute.org/hc/en-us/articles/360035890811-Resource-bundle): Stores GATK workflow specific plumbing, reference, and resources data. Example workspaces include:Somatic-SNVs-Indels-GATK4

### Downloading the `cnv_germline_pipeline` data bundle from 'gatk-best-practices'

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatkbestpractices', sas_token='sv=2020-04-08&si=prod&sr=c&sig=6SaDfKtXAIfdpO%2BkvNA%2FsTNmNij%2Byh%2F%2F%2Bf98WAUqs7I%3D')     
blob_service_client.get_blob_to_path('dataset/cnv_germline_pipeline', 'Homo_sapiens_assembly19.truncated.fasta', './Homo_sapiens_assembly19.truncated.fasta')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatkbestpractices', sas_token='sv=2020-04-08&si=prod&sr=c&sig=6SaDfKtXAIfdpO%2BkvNA%2FsTNmNij%2Byh%2F%2F%2Bf98WAUqs7I%3D')     
blob_service_client.get_blob_to_path('dataset/cnv_germline_pipeline', 'Homo_sapiens_assembly19.truncated.fasta.fai', './Homo_sapiens_assembly19.truncated.fasta.fai')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatkbestpractices', sas_token='sv=2020-04-08&si=prod&sr=c&sig=6SaDfKtXAIfdpO%2BkvNA%2FsTNmNij%2Byh%2F%2F%2Bf98WAUqs7I%3D')     
blob_service_client.get_blob_to_path('dataset/cnv_germline_pipeline', 'Homo_sapiens_assembly19.truncated.dict', './Homo_sapiens_assembly19.truncated.dict')

# DATASET 5. Downloading 'gatk-test-data' from Azure Genomic Data Lake

We will demonstrate how to download specific 'gatk-test-data' from Azure Genomics Data Lake. 


### Getting the Genomics data from Azure Genomics Data Lake

[Description of dataset](https://gatk.broadinstitute.org/hc/en-us/articles/360035890811-Resource-bundle): Additional public test data focusing on smaller data sets. For example, whole genome BAM, FASTQ, gVCF, VCF, etc. Example Workspaces include:Somatic-CNVs-GATK4.

### Downloading the `mutect2` data bundle from 'gatk-test-data'

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatktestdata', sas_token='sv=2020-04-08&si=prod&sr=c&sig=fzLts1Q2vKjuvR7g50vE4HteEHBxTcJbNvf%2FZCeDMO4%3D')     
blob_service_client.get_blob_to_path('dataset/mutect2', 'human_g1k_v37.20.21.fasta', './human_g1k_v37.20.21.fasta')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatktestdata', sas_token='sv=2020-04-08&si=prod&sr=c&sig=fzLts1Q2vKjuvR7g50vE4HteEHBxTcJbNvf%2FZCeDMO4%3D')     
blob_service_client.get_blob_to_path('dataset/mutect2', 'human_g1k_v37.20.21.fasta.fai', './human_g1k_v37.20.21.fasta.fai')

In [ ]:
import os
import uuid
import sys
from azure.storage.blob import BlockBlobService, PublicAccess

blob_service_client = BlockBlobService(account_name='datasetgatktestdata', sas_token='sv=2020-04-08&si=prod&sr=c&sig=fzLts1Q2vKjuvR7g50vE4HteEHBxTcJbNvf%2FZCeDMO4%3D')     
blob_service_client.get_blob_to_path('dataset/mutect2', 'human_g1k_v37.20.21.dict', './human_g1k_v37.20.21.dict')

**END OF NOTEBOOK**